# Learning With Errors (LWE)

## import libs

In [1]:
#!conda install pycrypto

In [2]:
import numpy as np
import pandas as pd
import scipy.stats
import math
import itertools
import random
from Crypto.Util import number

import matplotlib.pyplot as plt
from matplotlib import collections as matcoll

## Set vector s (secret)

Choose $s \in \mathbb{Z}^m_p$ with arbitrary $m \in \mathbb{N}$

In [3]:
#s = np.array([2, 3]) 
#s = np.array([10, 13, 9]) 
#s = np.array([10, 13, 9, 11]) 
#s = np.array([10, 13, 9, 11, 3]) 
s = np.array([10, 13, 9, 11, 3, 10, 13, 9, 11, 3, 10]) 
n = len(s)

## parameters

In [4]:
random.seed(42)  #set seed

# modulus 
#p = 17  # only prime numbers (it has to be a finite field)
p = number.getPrime(n.bit_length()**2, randfunc=np.random.bytes)  # using pycrypto lib  (p = O(n^2))
print("Prime:", p)

#size parameter
m = n
print('Count of equations:', m)

Prime: 47293
Count of equations: 11


## Construct the LWE problem without error

#### Construct A, b

In [5]:
A = np.random.randint(0, p, size=(m, n))

b = (np.matmul(A, s))%p  # system of linear equations without perturbation

## Solving

### Modified Gaussian Elimination

In [6]:
# Iterative Algorithm (xgcd)
def iterative_egcd(a, b):
    x,y, u,v = 0,1, 1,0
    while a != 0:
        q,r = b//a,b%a; m,n = x-u*q,y-v*q # use x//y for floor "floor division"
        b,a, x,y, u,v = a,r, u,v, m,n
    return b, x, y

def modinv(a, m):
    g, x, y = iterative_egcd(a, m) 
    if g != 1:
        return None
    else:
        return x % m
    
def solve_linear_congruence(a, b, m):
    """ Describe all solutions to ax = b  (mod m), or raise ValueError. """
    g = math.gcd(a, m)
    if b % g:
        raise ValueError("No solutions")
    a, b, m = a//g, b//g, m//g
    return modinv(a, m) * b % m, m

def print_solutions(a, b, m):
    print(f"Solving the congruence: {a}x = {b}  (mod {m})")
    x, mx = solve_linear_congruence(a, b, m)
    
    print(f"Particular solution: x = {x}")
    print(f"General solution: x = {x}  (mod {mx})")
    
# for debug
print_solutions(272, 256, 1009)

Solving the congruence: 272x = 256  (mod 1009)
Particular solution: x = 179
General solution: x = 179  (mod 1009)


In [7]:
def gaussianEliminationForward(A, b, modulus):
    (m, n) = A.shape
    
    A = np.copy(A[:n][:])
    b = np.copy(b[:n])
    
    
    for j in range(n):  # quadratic matrix
        i = j
        while(i<n-1):
            rowUpper = A[i, :]
            rowUpperLeader = rowUpper[j]
            leftUpper = b[i]
            rowLower = A[i+1, :]
            rowLowerLeader = rowLower[j]
            leftLower = b[i+1]

            if rowLowerLeader==0:
                pass
            elif rowUpperLeader==0 and rowLowerLeader!=0:
                # swap rows
                A[[i, i+1]] = A[[i+1, i]]
                b[[i, i+1]] = b[[i+1, i]]
                i=j-1  # redo column
                
            elif rowUpperLeader!=0 and rowLowerLeader!=0:
                lcm = np.lcm(rowUpperLeader, rowLowerLeader)
                rowLowerNew = (lcm/rowLowerLeader)*rowLower - (lcm/rowUpperLeader)*rowUpper
                leftLowerNew = (lcm/rowLowerLeader)*leftLower - (lcm/rowUpperLeader)*leftUpper
                
                A[i+1, :] = rowLowerNew%modulus
                b[i+1] = leftLowerNew%modulus
                
            i+=1

    return A, b



    
def gaussianEliminationBackward(A, b, modulus):
    (m, n) = A.shape
    x = np.zeros(m)
    
    for i in range(n-1, -1, -1):
        equLeft = A[i, :]
        equLeftCoef = equLeft[i]
        equRight = b[i]
        equRightCoef = equRight - np.dot(x, equLeft)
                    
        solution, mx = solve_linear_congruence(equLeftCoef, equRightCoef, modulus)
        x[i] = solution
    
    return x

# for debug
#print(A[:n])
A_new, b_new = gaussianEliminationForward(A, b, p)
#print(A_new)
#print()
#print(b[:n].astype(int))
#print(b_new.astype(int))
#print()
#print(scipy.linalg.solve(A[:m], b[:m]))
#print(scipy.linalg.solve(A_new, b_new))

In [8]:
try:
    A_new, b_new = gaussianEliminationForward(A, b, p)
    x = gaussianEliminationBackward(A_new%p, b_new%p, p)
    print("Guess:", x.astype(int)%p, "\t", "Right Solution:", s)
except ValueError:  # occurs by linear dependency in the matrix subsetA
    print("linear dependency")

Guess: [10 13  9 11  3 10 13  9 11  3 10] 	 Right Solution: [10 13  9 11  3 10 13  9 11  3 10]


sometimes I got wrong solution because of integer overflow. Particularly with big p